In [1]:
import numpy as np
import os
from importlib import reload
from copy import deepcopy
import json
import pandas as pd
from tqdm.notebook import tqdm

import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler

import torch
from torch import nn
import pytorch_lightning as pl

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from ray.tune.integration.pytorch_lightning import TuneReportCallback, TuneReportCheckpointCallback

seed_everything(42)

Global seed set to 42


42

## data

In [2]:
import src.data
reload(src.data)
from src.data import CSIDataset

from torch.utils.data import DataLoader

## Model

In [3]:
import src.model
reload(src.model)
from src.model import CSIModel


def experiment(args):
    dataset = 'weibo'
    path = f'/media/external_10TB/10TB/ramezani/Omranpour/assets/{dataset}/'
    
    train_set = CSIDataset(pkl_dir=path + 'train/pkls/')
    val_set = CSIDataset(pkl_dir=path + 'validation/pkls/')
    train_loader = DataLoader(train_set, batch_size=1, shuffle=False, num_workers=1)
    val_loader = DataLoader(val_set, batch_size=1, shuffle=False, num_workers=1)
    
    
    conf = {
        'capture_input_dim' : 112,
        'score_input_dim' : 50,
        'd_Wa': 100,
        'd_lstm' : 50,
        'd_Wr' : 100,
        'd_Wu' : 100,
        'd_Ws' : 1,
        'lr': args['lr'],
        'dropout' : args['dropout'],
        'weight_decay' : args['weight_decay']
    }
    model = CSIModel(conf)

    name = f"dataset={dataset}-do={args['dropout']}-lr={args['lr']}-wd={args['weight_decay']}"
    save_dir = f'/media/external_10TB/10TB/ramezani/Omranpour/CSI/weights/{name}/'
    logger = TensorBoardLogger(save_dir='logs/', name=name)
    checkpoint = ModelCheckpoint(
        dirpath=save_dir, 
        filename='{epoch}-{val_loss:.2f}', 
        monitor='val_loss',
        mode='min',
        save_top_k=10, 
        every_n_epochs = 5
    )
#     reporter = TuneReportCallback(
#         {
#             "loss": "ptl/val_loss",
#             "mean_accuracy": "ptl/val_acc"
#         },
#         on="validation_end"
#     )
    os.makedirs(save_dir, exist_ok=True)
    json.dump(conf, open(save_dir + 'config.json', 'w'))

    trainer = Trainer(
        benchmark=True, 
        gpus=[1], 
        accumulate_grad_batches=64,
        logger=logger, 
        enable_progress_bar=False,
        max_epochs=10,
        callbacks=[checkpoint]
    )
    trainer.fit(model, train_loader, val_loader)
    res = trainer.validate(val_loader)[0]
    tune.report(**res)

In [ ]:
analysis = tune.run(
    experiment,
    num_samples=4,
    resources_per_trial={"cpu": 1, "gpu": 2},
    verbose=1,
    config={
        "weight_decay": tune.grid_search([0., 0.1, 0.01, 0.001]),
        "lr": tune.loguniform(1e-5, 1e-1),
        "dropout": tune.uniform(0., 0.3)
    }
)


In [ ]:
df = analysis.results_df